In [29]:
import wrds
db=wrds.Connection(wrds_username='rabdalla')
#db.create_pgpass_file()

Loading library list...
Done


## 2 - Data download and cleaning

In [1]:
import numpy as np
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [30]:
query_crsp_vw = """
SELECT date, vwretd
FROM crsp.msi
WHERE date BETWEEN '2000-01-01' AND '2024-12-31'
"""

crsp_vw = db.raw_sql(query_crsp_vw, date_cols=['date'])
crsp_vw.to_csv("crsp_value_weighted_returns.csv", index=False)

In [2]:
USD_monthly_returns = pd.read_csv('data/US_value_weighted_returns.csv', parse_dates=['date'], index_col='date')
USD_monthly_returns.head()

,vwretd
date,
2000-01-31,-0.039622
2000-02-29,0.031760
2000-03-31,0.053499
2000-04-28,-0.059519
2000-05-31,-0.038863


In [5]:
# Japan
japan = pd.read_csv('data/JAPAN.csv', parse_dates=['date'], index_col='date')
japan = japan[['mportret']]
# japan = japan[japan.index >= '2002-04-01']

# Australia
australia = pd.read_csv('data/AUSTRALIA.csv', parse_dates=['date'], index_col='date')
australia = australia[['mportret']]
# australia = australia[australia.index >= '2002-04-01']

# Germany
germany = pd.read_csv('data/GERMANY.csv', parse_dates=['date'], index_col='date')
germany = germany[['mportret']]
# germany = germany[germany.index >= '2002-04-01']

# France
france = pd.read_csv('data/FRANCE.csv', parse_dates=['date'], index_col='date')
france = france[['mportret']]
# france = france[france.index >= '2002-04-01']

# Switzerland
switzerland = pd.read_csv('data/SWITZERLAND.csv', parse_dates=['date'], index_col='date')
switzerland = switzerland[['mportret']]
# switzerland = switzerland[switzerland.index >= '2002-04-01']

# United Kingdom
unitedkingdom = pd.read_csv('data/UNITEDKINGDOM.csv', parse_dates=['date'], index_col='date')
unitedkingdom = unitedkingdom[['mportret']]
# unitedkingdom = unitedkingdom[unitedkingdom.index >= '2002-04-01']

switzerland.head()

,mportret
date,
2000-01-31,-0.072023
2000-02-29,-0.001949
2000-03-31,0.081995
2000-04-28,0.002559
2000-05-31,0.042735


In [6]:
# Japan: **JPY per 1 USD** (i.e. USD → JPY)
fx_jpus = pd.read_csv('data/EXJPUS.csv', parse_dates=['observation_date'], index_col='observation_date')
# Example: 1 USD = 153.81 JPY → USD is the base

# Switzerland: **CHF per 1 USD** (i.e. USD → CHF)
fx_szus = pd.read_csv('data/EXSZUS.csv', parse_dates=['observation_date'], index_col='observation_date')

# Australia: **USD per 1 AUD** (i.e. AUD → USD)
fx_usal = pd.read_csv('data/EXUSAL.csv', parse_dates=['observation_date'], index_col='observation_date')
# Example: 1 AUD = 0.66 USD → AUD is the base

# Eurozone (France, Germany): **USD per 1 EUR** (i.e. EUR → USD)
fx_eu = pd.read_csv('data/EXUSEU.csv', parse_dates=['observation_date'], index_col='observation_date')
# Example: 1 EUR = 1.08 USD → EUR is the base

# United Kingdom: **USD per 1 GBP** (i.e. GBP → USD)
fx_uk = pd.read_csv('data/EXUSUK.csv', parse_dates=['observation_date'], index_col='observation_date')
# Example: 1 GBP = 1.25 USD → GBP is the base

fx_szus.head()



,EXSZUS
observation_date,
2000-01-01,1.5903
2000-02-01,1.6348
2000-03-01,1.6636
2000-04-01,1.6657
2000-05-01,1.7190


truncating indices for fx and returns to align:

In [7]:
# Country indices
australia.index     = australia.index.to_period('M').to_timestamp()
germany.index       = germany.index.to_period('M').to_timestamp()
france.index        = france.index.to_period('M').to_timestamp()
switzerland.index   = switzerland.index.to_period('M').to_timestamp()
unitedkingdom.index = unitedkingdom.index.to_period('M').to_timestamp()
japan.index         = japan.index.to_period('M').to_timestamp()

# FX rates
fx_usal.index = fx_usal.index.to_period('M').to_timestamp()
fx_eu.index   = fx_eu.index.to_period('M').to_timestamp()
fx_szus.index = fx_szus.index.to_period('M').to_timestamp()
fx_uk.index   = fx_uk.index.to_period('M').to_timestamp()
fx_jpus.index = fx_jpus.index.to_period('M').to_timestamp()

data_aus = australia.join(fx_usal, how='inner')
data_jpn = japan.join(fx_jpus, how='inner')
data_ger = germany.join(fx_eu, how='inner')
data_fra = france.join(fx_eu, how='inner')
data_chn = switzerland.join(fx_szus, how='inner')
data_uk  = unitedkingdom.join(fx_uk, how='inner')

data_aus.rename(columns={'mportret': 'returns', 'EXUSAL': 'fx_rate'}, inplace=True)
data_jpn.rename(columns={'mportret': 'returns', 'EXJPUS': 'fx_rate'}, inplace=True)
data_ger.rename(columns={'mportret': 'returns', 'EXUSEU': 'fx_rate'}, inplace=True)
data_fra.rename(columns={'mportret': 'returns', 'EXUSEU': 'fx_rate'}, inplace=True)
data_chn.rename(columns={'mportret': 'returns', 'EXSZUS': 'fx_rate'}, inplace=True)
data_uk.rename(columns={'mportret': 'returns', 'EXUSUK': 'fx_rate'}, inplace=True)



## 3.a. - returns of each index in USD

In [8]:
# Compute FX return: % change in USD per AUD
data_aus['fx_return'] = data_aus['fx_rate'] / data_aus['fx_rate'].shift(1) - 1

# Compute USD return from local return and FX return
data_aus['usd_return'] = (1 + data_aus['returns']) * (1 + data_aus['fx_return']) - 1

data_aus.dropna(inplace=True)

data_uk['fx_return'] = data_uk['fx_rate'] / data_uk['fx_rate'].shift(1) - 1
data_uk['usd_return'] = (1 + data_uk['returns']) * (1 + data_uk['fx_return']) - 1
data_uk.dropna(inplace=True)

data_ger['fx_return'] = data_ger['fx_rate'] / data_ger['fx_rate'].shift(1) - 1
data_ger['usd_return'] = (1 + data_ger['returns']) * (1 + data_ger['fx_return']) - 1   
data_ger.dropna(inplace=True)

data_fra['fx_return'] = data_fra['fx_rate'] / data_fra['fx_rate'].shift(1) - 1
data_fra['usd_return'] = (1 + data_fra['returns']) * (1 + data_fra['fx_return']) - 1
data_fra.dropna(inplace=True) 

data_fra.tail()


,returns,fx_rate,fx_return,usd_return
date,,,,
2024-08-01,0.017771,1.1020,0.015949,0.034003
2024-09-01,0.008740,1.1104,0.007623,0.016429
2024-10-01,-0.051356,1.0895,-0.018822,-0.069211
2024-11-01,-0.021042,1.0621,-0.025149,-0.045662
2024-12-01,0.035233,1.0472,-0.014029,0.020710


In [10]:
# Invert CHF/USD → to USD/CHF, same for yen
data_chn['fx_rate'] = 1 / data_chn['fx_rate']
data_chn['fx_return'] = data_chn['fx_rate'] / data_chn['fx_rate'].shift(1) - 1
data_chn['usd_return'] = (1 + data_chn['returns']) * (1 + data_chn['fx_return']) - 1
data_chn.dropna(inplace=True)

data_jpn['fx_rate'] = 1 / data_jpn['fx_rate']
data_jpn['fx_return'] = data_jpn['fx_rate'] / data_jpn['fx_rate'].shift(1) - 1
data_jpn['usd_return'] = (1 + data_jpn['returns']) * (1 + data_jpn['fx_return']) - 1
data_jpn.dropna(inplace=True)

data_chn.head()


,returns,fx_rate,fx_return,usd_return
date,,,,
2000-02-01,-0.001949,1.6348,0.027982,0.025979
2000-03-01,0.081995,1.6636,0.017617,0.101056
2000-04-01,0.002559,1.6657,0.001262,0.003825
2000-05-01,0.042735,1.7190,0.031999,0.076101
2000-06-01,0.016003,1.6420,-0.044793,-0.029507


merging all in one dataframe

In [14]:
USD_monthly_returns = USD_monthly_returns.rename(columns={'vwretd': 'USA'})
USD_monthly_returns.index = USD_monthly_returns.index.to_period('M').to_timestamp()

# Create unified DataFrame with all countries' USD returns
panel = pd.concat([
    data_aus['usd_return'].rename('Australia'),
    data_jpn['usd_return'].rename('Japan'),
    data_ger['usd_return'].rename('Germany'),
    data_fra['usd_return'].rename('France'),
    data_chn['usd_return'].rename('Switzerland'),
    data_uk['usd_return'].rename('UK'),
    USD_monthly_returns['USA']
], axis=1)

panel = panel.dropna()
panel.to_csv("data/panel.csv")
panel.head()

,Australia,Japan,Germany,France,Switzerland,UK,USA
date,,,,,,,
2000-03-01,-0.015016,-0.034531,-0.020870,-0.002063,0.101056,0.051703,0.053499
2000-04-01,-0.014769,-0.042764,-0.064312,-0.013842,0.003825,-0.017993,-0.059519
2000-05-01,-0.029129,-0.050561,-0.067036,-0.027612,0.076101,-0.037558,-0.038863
2000-06-01,0.086921,0.026169,0.014415,0.052475,-0.029507,0.010025,0.051576
2000-07-01,-0.010310,-0.071912,0.010317,0.010210,0.046896,0.010366,-0.017679


## 3.a. Using aligned daily exchange rates

In [15]:
# 🇯🇵 Japan: JPY per 1 USD → USD is the base → must invert
fx_jpy_usd = pd.read_csv('data/DEXJPUS.csv', parse_dates=['observation_date'], index_col='observation_date')
# Example: 1 USD = 153.81 JPY → USD is the base

# 🇨🇭 Switzerland: CHF per 1 USD → USD is the base → must invert
fx_chf_usd = pd.read_csv('data/DEXSZUS.csv', parse_dates=['observation_date'], index_col='observation_date')
# Example: 1 USD = 0.90 CHF → USD is the base

# 🇦🇺 Australia: USD per 1 AUD → AUD is the base → no inversion
fx_usd_aud = pd.read_csv('data/DEXUSAL.csv', parse_dates=['observation_date'], index_col='observation_date')
# Example: 1 AUD = 0.66 USD → AUD is the base

# 🇪🇺 Eurozone: USD per 1 EUR → EUR is the base → no inversion
fx_usd_eur = pd.read_csv('data/DEXUSEU.csv', parse_dates=['observation_date'], index_col='observation_date')
# Example: 1 EUR = 1.08 USD → EUR is the base

# 🇬🇧 United Kingdom: USD per 1 GBP → GBP is the base → no inversion
fx_usd_gbp = pd.read_csv('data/DEXUSUK.csv', parse_dates=['observation_date'], index_col='observation_date')
# Example: 1 GBP = 1.25 USD → GBP is the base

In [16]:
# Convert 'VALUE' column name to consistent 'fx_rate'
fx_jpy_usd.rename(columns={'DEXJPUS': 'fx_rate'}, inplace=True)
fx_chf_usd.rename(columns={'DEXSZUS': 'fx_rate'}, inplace=True)
fx_usd_aud.rename(columns={'DEXUSAL': 'fx_rate'}, inplace=True)
fx_usd_eur.rename(columns={'DEXUSEU': 'fx_rate'}, inplace=True)
fx_usd_gbp.rename(columns={'DEXUSUK': 'fx_rate'}, inplace=True)

# Forward-fill missing values (FRED has NaNs on weekends/holidays)
for fx in [fx_jpy_usd, fx_chf_usd, fx_usd_aud, fx_usd_eur, fx_usd_gbp]:
    fx['fx_rate'] = fx['fx_rate'].ffill()


fx_usd_gbp.head()

,fx_rate
observation_date,
2000-01-03,1.6270
2000-01-04,1.6370
2000-01-05,1.6415
2000-01-06,1.6475
2000-01-07,1.6384


In [18]:
# Japan
japan = pd.read_csv('data/JAPAN.csv', parse_dates=['date'], index_col='date')
japan = japan[['mportret']]

# Australia
australia = pd.read_csv('data/AUSTRALIA.csv', parse_dates=['date'], index_col='date')
australia = australia[['mportret']]

# Germany
germany = pd.read_csv('data/GERMANY.csv', parse_dates=['date'], index_col='date')
germany = germany[['mportret']]

# France
france = pd.read_csv('data/FRANCE.csv', parse_dates=['date'], index_col='date')
france = france[['mportret']]

# Switzerland
switzerland = pd.read_csv('data/SWITZERLAND.csv', parse_dates=['date'], index_col='date')
switzerland = switzerland[['mportret']]

# United Kingdom
unitedkingdom = pd.read_csv('data/UNITEDKINGDOM.csv', parse_dates=['date'], index_col='date')
unitedkingdom = unitedkingdom[['mportret']]


In [19]:
# Australia — daily data using DEX
data_aus_dex = australia.join(fx_usd_aud, how='inner')
data_aus_dex.rename(columns={'mportret': 'returns'}, inplace=True)

# Japan
data_jpn_dex = japan.join(fx_jpy_usd, how='inner')
data_jpn_dex.rename(columns={'mportret': 'returns'}, inplace=True)
data_jpn_dex['fx_rate'] = 1 / data_jpn_dex['fx_rate']

# Germany
data_ger_dex = germany.join(fx_usd_eur, how='inner')
data_ger_dex.rename(columns={'mportret': 'returns'}, inplace=True)

# France
data_fra_dex = france.join(fx_usd_eur, how='inner')
data_fra_dex.rename(columns={'mportret': 'returns'}, inplace=True)

# Switzerland
data_chn_dex = switzerland.join(fx_chf_usd, how='inner')
data_chn_dex.rename(columns={'mportret': 'returns'}, inplace=True)
data_chn_dex['fx_rate'] = 1 / data_chn_dex['fx_rate']

# United Kingdom
data_uk_dex = unitedkingdom.join(fx_usd_gbp, how='inner')
data_uk_dex.rename(columns={'mportret': 'returns'}, inplace=True)

In [21]:
# Australia (no inversion was needed)
data_aus_dex['fx_return'] = data_aus_dex['fx_rate'] / data_aus_dex['fx_rate'].shift(1) - 1
data_aus_dex['usd_return'] = (1 + data_aus_dex['returns']) * (1 + data_aus_dex['fx_return']) - 1

# Japan
data_jpn_dex['fx_return'] = data_jpn_dex['fx_rate'] / data_jpn_dex['fx_rate'].shift(1) - 1
data_jpn_dex['usd_return'] = (1 + data_jpn_dex['returns']) * (1 + data_jpn_dex['fx_return']) - 1

# Germany
data_ger_dex['fx_return'] = data_ger_dex['fx_rate'] / data_ger_dex['fx_rate'].shift(1) - 1
data_ger_dex['usd_return'] = (1 + data_ger_dex['returns']) * (1 + data_ger_dex['fx_return']) - 1

# France
data_fra_dex['fx_return'] = data_fra_dex['fx_rate'] / data_fra_dex['fx_rate'].shift(1) - 1
data_fra_dex['usd_return'] = (1 + data_fra_dex['returns']) * (1 + data_fra_dex['fx_return']) - 1

# Switzerland
data_chn_dex['fx_return'] = data_chn_dex['fx_rate'] / data_chn_dex['fx_rate'].shift(1) - 1
data_chn_dex['usd_return'] = (1 + data_chn_dex['returns']) * (1 + data_chn_dex['fx_return']) - 1

# United Kingdom
data_uk_dex['fx_return'] = data_uk_dex['fx_rate'] / data_uk_dex['fx_rate'].shift(1) - 1
data_uk_dex['usd_return'] = (1 + data_uk_dex['returns']) * (1 + data_uk_dex['fx_return']) - 1

data_uk_dex.tail()

,returns,fx_rate,fx_return,usd_return
2024-08-30,0.008425,1.3132,0.022741,0.031358
2024-09-30,-0.017154,1.3399,0.020332,0.002829
2024-10-31,-0.017104,1.2856,-0.040525,-0.056936
2024-11-29,0.026111,1.2699,-0.012212,0.013580
2024-12-31,-0.011381,1.2521,-0.014017,-0.025239


In [23]:
dex_panel = pd.concat([
    data_aus_dex['usd_return'].rename('Australia'),
    data_jpn_dex['usd_return'].rename('Japan'),
    data_ger_dex['usd_return'].rename('Germany'),
    data_fra_dex['usd_return'].rename('France'),
    data_chn_dex['usd_return'].rename('Switzerland'),
    data_uk_dex['usd_return'].rename('UK')
], axis=1)

dex_panel.dropna(inplace=True)
dex_panel.to_csv("data/dex_panel.csv")
dex_panel.head()


,Australia,Japan,Germany,France,Switzerland,UK
2000-02-29,-0.055022,-0.009572,0.115900,0.077583,-0.012130,-0.014806
2000-03-31,-0.000608,0.062596,-0.008621,0.010421,0.084272,0.074667
2000-04-28,-0.030008,-0.083439,-0.093474,-0.044577,-0.033475,-0.041776
2000-05-31,-0.021696,-0.072107,-0.001282,0.040921,0.065402,-0.029204
2000-06-30,0.105466,0.063183,-0.010693,0.026425,0.051246,0.020822
